### Prototype Code : Prompt Generator

In [ ]:
# Prompt Generator + LLM

# CONTEXT
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case.

Here is the UI element (some elements have a link attached to them): {question}
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"

DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

"""

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)
    # Model
    model = OllamaLLM(model=model_str)
    # Chain
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, chain, model_str : str = "llama3.1" , template : str = template, batch_size : int = 10):
    
    return_data = []
    
    for sub_data in data:
        element_test_cases = []
        i = 0
        j = 0
        print(f"Batch Number: {ceil(len(sub_data)/batch_size)}")
        while (j<ceil(len(sub_data)/batch_size)):
            print(f"[{j}] Batch {str(len(sub_data[i:i+batch_size]))}")
            appending = []
            for dat in sub_data[i:i+batch_size]:
                appending.append(chain.invoke({"question": str(dat)}))
            element_test_cases.append(appending)
            i+=batch_size
            j+=1
            prompt = ChatPromptTemplate.from_template(template)
            model = OllamaLLM(model=model_str)
            chain = prompt | model
        return_data.append(element_test_cases)
    return return_data


### Version 1 : Dissection and Simplification

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# CONTEXT V1
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case for usability.

Some UI elements have a link attached to them and other properties.
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

Here is the UI element : {question}
"""

# Prompt Generator + LLM
# TODO : Experiment With Prompts
# TODO : Create Altered Version For Fine-Tuned Model Version (aka System Proper)

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    for item in data:
        return_data.append(chain.invoke({"question": str(item)}))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)

    return return_data
